# Part 1  - Replica Exchange Parallel Tempering

### In class we derived the equations for Parallel Tempering, i.e. we run several simulations at different temperatures and exchange configurations between them in order to get over energy barriers. Here we will try it out for a simple 2 well system. Normally, to be useful these simulations would have to run _in parallel_, however it is not a requirement, and here we will run each 'replica' separately.



### Run the next cell to define the Langevin Dynamics simulation function

In [ ]:
%pylab inline

#this function returns the potential energy and force
def double_well(x,x0=2,A=0.7,B=0.0):
    potential = A*((x-x0)**2)*((x+x0)**2) - B*x
    potential = potential #- np.min(potential)
    force = -( 4 * A*(x**3) - 4*A*(x0**2)*x - B)
    #hessian = 12 *A * (x**2) - 4*A*(x0**2)
    
    return potential, force

#this is step A
def position_update(x,v,dt):
    x_new = x + v*dt/2.
    return x_new

#this is step B
def velocity_update(v,F,dt):
    v_new = v + F*dt/2.
    return v_new

def random_velocity_update(v,gamma,kBT,dt):
    R = np.random.normal(size=np.size(v))
    c1 = np.exp(-gamma*dt)
    c2 = np.sqrt(1-c1*c1)*np.sqrt(kBT)
    v_new = c1*v + R*c2
    return v_new

def baoab(potential, max_time, dt, gamma, kBT, initial_position, initial_velocity,
                                        save_frequency=5, **kwargs ):
    x = initial_position
    v = initial_velocity
    t = 0
    step_number = 0
    positions = []
    velocities = []
    total_energies = []
    potential_energies = []
    save_times = []
    
    while(t<max_time):
        
        # B
        potential_energy, force = potential(x,**kwargs)
        v = velocity_update(v,force,dt)
        
        #A
        x = position_update(x,v,dt)

        #O
        v = random_velocity_update(v,gamma,kBT,dt)
        
        #A
        x = position_update(x,v,dt)
        
        # B
        potential_energy, force = potential(x,**kwargs)
        v = velocity_update(v,force,dt)
        
        if step_number%save_frequency == 0 and step_number>0:
            e_total = .5*v*v + potential_energy

            positions.append(x)
            velocities.append(v)
            total_energies.append(e_total)
            potential_energies.append(potential_energy)
            save_times.append(t)
        
        t = t+dt
        step_number = step_number + 1
    
    return save_times, positions, velocities, total_energies, potential_energies  



### Play around with the parameters A, B, and x_0 to see how they effect the potential we will be simulating with. 
- Default parameters set above which make sense for this exercise are $A=0.7$, $B=0.0$, $x_0=2$, so those have been set as defaults above

### Question 1: Explain in your own words how these parameters change the potential:

*your answer to question 1 here!*

In [ ]:
my_A = 0.7
my_B = 0.0
my_x0 = 2.0
pmax = my_A*(my_x0**4)
xpoints = np.arange(-4*my_x0,4*my_x0,0.1)

potential, force = double_well(xpoints, my_x0, my_A, my_B)

plt.plot(xpoints,potential-potential.min(),label="$U(x)$",c='red')
plt.plot(xpoints,force,label="$U'(x)$",c='blue')

plt.axhline(0,ls='--',c='black')

plt.ylim(-2*pmax,2*pmax)
plt.xlabel("$x$")
plt.legend(loc=0)

### The next cell gives an example running the system at one temperature

### Question 2: 
Run the simulation with different parameters, but especially changing the temperature. I found good results using $\gamma=1$ and $dt=0.1$.

a) Write a function to compute the ideal distribution for position for this system and plot it. $P_{ideal}(x) = e^{-\beta U(x)}/Z$, $Z=\int dx e^{-\beta U(X)}$. You can use something like [trapz](https://numpy.org/doc/stable/reference/generated/numpy.trapz.html) to integrate/normalize this histogram or write your own function to compute $\int dx f(x) \approx \sum_{i=1}^N f(x_i) \Delta x$.

b) Histogram and plot the histogram of position from the `positions` output by Langevin dynamics

c) Explain in your own words the effect of changing temperature (my_kBT)

d) Use this information to answer the question after, is sampling at my_kBT=1.0 ergodic (does it sample all accessible states of the system?) Why or why not?

In [ ]:
my_kBT = 1.0

my_gamma=1.0
my_dt=0.1

initial_position = -2
initial_velocity = 0.0
my_max_time = 500

times, positions, velocities, total_energies, potential_energies = baoab(double_well, \
                                                                            my_max_time, my_dt, my_gamma, my_kBT, \
                                                                            initial_position, initial_velocity,save_frequency=5)

plt.plot(times,positions,marker='',label='position',linestyle='-')

xlabel('time')
legend(loc='upper center')

plt.figure()
plt.plot(times,potential_energies,marker='o',linestyle='',label='Simulated E')
xlabel('time')
ylabel("Potential Energy")
legend()

Note, it is possible to run multiple simulations at once the way the Langevin dynamics code is written, but you don't have to use that feature since it is more complicated. Example in next cell

In [ ]:
my_kBT = [1.0,1.0]

my_gamma=1.0
my_dt=0.1

initial_position = np.array([-2,2])
initial_velocity = np.array([0.0,0.0])
my_max_time = 500

times, positions, velocities, total_energies, potential_energies = baoab(double_well, \
                                                                            my_max_time, my_dt, my_gamma, my_kBT, \
                                                                            initial_position, initial_velocity,save_frequency=5)

positions = np.array(positions)
plt.plot(times,positions[:,0],label="Start x=-2")
plt.plot(times,positions[:,1],label="Start x=2")


xlabel('time')
legend(loc=0)

#### Your answer to question 2 here

# Problem 3: Parallel Tempering

- Now write a code that performs parallel tempering using the algorithm below for some set of temperatures 
- During the algorithm, compute and print the acceptance ratio for swapping each replica
- If you can, keep track of the temperature of each replica, and plot it as a function of time (example below)
- At the end, plot a histogram of positions from samples at each temperature and compare to the exact distribution; my results for the above double well paramters and temperatures $k_B T=\{1,3,6,9\}$ are below
- Tips:
 - In my simulations, I first equilibrated for $t=100$ at teach temperature before starting the replica exchange
 - I ended up attempting an exchange every 50 MD steps (with dt = 0.1)
 - I ran until I had made 10000 swap attempts
 - Compute statistics from samples at a particular temperature, not in a particular replica, which moves up and down. It's easier to generate $N$ trajectories at the $N$ temperatures from the first place during the algorithm, but you will have to figure out the best way to do that part 
 - If you get memory errors, try increasing `save_frequency` (i.e. save configurations and energies less often), it probably doesn't have to be too small
 - For my results, the code only took a few minutes to run. Still, try testing by running much shorter simulations that only take a few seconds, then run longer to get final results


## Algorithm

- Make $N$ copies of of your system.
- For system $i$, run Langevin dynamics at temperature $T_i$ for $n$ steps
- Pick a random integer $j$ from 0 to $N-2$, and decide whether to swap replicas $j$ and $j+1$ using the critereon $P_{accept} = \min(1,\exp(\Delta U \Delta (1/T)))$ where $\Delta U=U_{j+1} - U_j$ and $\Delta (1/T) = \frac{1}{T_{j+1}} - \frac{1}{T_j}$. (See notes from class)
- If accepted, swap the two new configurations (positions and velocities) for that time, otherwise do nothing

**Exchange example**
<img src="figures/2020/exchange_plot_example.jpg" width=300px height=300px />
**Distribution example**
<img src="figures/2020/replica_exchange_probability_comparison.jpg" width=300px height=300px />